<a href="https://colab.research.google.com/github/beneDesp/GLDM3002/blob/lab2/mattWalsh_LAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
#/content/Cereals.csv
cereals_df = pd.read_csv('/content/Cereals.csv')

# Regression

**Use Python** to load your dataset ("Cereals.csv") from your Canvas Data folder, to predict model form the data
as follows:

**(100%) Regression**

Predict Multiple Regression Model for target or response variable (**`rating`**) vs. input or independent variables
(**`calories, protein, fat, sodium, and fiber`**)

In [ ]:
cereals_df.dtypes

## <u> Subquestions </u>

**a)** ***quantitative/numeric varables:*** calories, protein, fat, sodium, fiber, carbo, sugars, potass, vitamins, weight, cups, rating

**b.)**

***nominal variables:*** name, mfr

***ordinal variables:*** type (ordered by temperature), shelf (although identified as an integer, the integer represents a non-continuous distance from ground)

# Summary Statistics

In [ ]:
#type(cereals_df.describe())
#cereals_df.describe()
cols = ['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'weight', 'cups', 'rating']
print(cereals_df[cols].describe().loc[['mean','50%','min','max','std']])

# Plot Histogram

In [ ]:
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
cols = ['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'weight', 'cups', 'rating']
scaled_cereals_df= pd.DataFrame(sc.fit_transform(cereals_df[cols]), columns=cols)

print(scaled_cereals_df.hist(figsize=(15, 10)))

## <u> Additional tests for subquestions below: </u>

In [ ]:
#print(type(scaled_cereals_df))
#print(type(cereals_df))
print("Describe scaled variables output:")
print(scaled_cereals_df.describe()) #.loc[['mean','50%','min','max','std']])

def find_iqr(x):
  return np.subtract(*np.nanpercentile(x, [75, 25]))
print("IQR output:")
print(scaled_cereals_df.apply(find_iqr))
print("Skew output:")
print(scaled_cereals_df.skew(axis=0))

## <u> Subquestions </u>

**a)** ***Varables with largest variablilty:***


1.   calories (fat tails, despite strong central tendency)
2.   per Interquartile range (IQR) of scaled variables, fat and sugars are the variables with the largest variablility

**b.)** ***Seemingly-skewed variables:*** fiber, potassium, vitamins rate high on a measure of skewedness

**c.)** ***Seemingly-extreme variables:*** potassium and fat have the seemingly most extreme outlier values as indicated by the range of their x-axis in the plot of histograms where all variables are scaled

# Use Boxplot

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")
#tips = sns.load_dataset("tips")
ax = sns.boxplot(x="shelf", y="rating",
data=cereals_df, palette="Set3")

# Correlation

## <u> Subquestions </u>

**a)** ***Correlation table:***

In [ ]:
cols = ['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'weight', 'cups', 'rating']
cereals_df[cols].corr()

**b.)** ***Discuss***

The pair of vairiables most strongly correlated are potassium and fiber  and they are positively correlated.

In [1]:
import matplotlib.pyplot as plt

matrix = cereals_df[cols].corr().round(2)
sns.heatmap(matrix, annot=True)
plt.show()

NameError: name 'cereals_df' is not defined